In [ ]:
from edward.models import Normal
import numpy as np
import tensorflow as tf

In [2]:
# --- manually!

In [3]:
t = 0.1
delta = 10e-3
eta = 1e-1

vars_manual = [1., 2.]
grads_manual = [3.1018744, 1.5509372]
s_n_manual = [0., 0.]
n_manual = 1

In [14]:
# grads_manual = [2.7902498, 1.241244]
grads_manual = [2.6070995, 1.0711095]
n_manual = 3

In [15]:
print(n_manual)

for i in [0, 1]:
    s_n_manual[i] = (t * grads_manual[i]**2) + (1 - t)*s_n_manual[i]
    print('s_n[i]:', s_n_manual[i])
    p_n_first = (eta * n_manual**(-.5 + delta))
    p_n_second = (1 + np.sqrt(s_n_manual[i]))**(-1)
    p_n = p_n_first * p_n_second
    print('p_n_first:', p_n_first)
    print('p_n_second:', p_n_second)
    print('p_n:', p_n)
    
    vars_manual[i] += -p_n * grads_manual[i]
    print('var:', vars_manual[i])
    
    print('')

3
s_n[i]: 2.1597428437294326
p_n_first: 0.05837280797536004
p_n_second: 0.404922832044
p_n: 0.0236364827198
var: 0.694741731383

s_n[i]: 0.44822725824311604
p_n_first: 0.05837280797536004
p_n_second: 0.598982532688
p_n: 0.0349642923612
var: 1.80355358733



In [ ]:
# tensorflow...

In [33]:
t = 0.1
delta = 10e-3
eta = 1e-1
    
def alp_optimizer_apply_gradients(n, s_n, grads_and_vars):
    ops = []
    for i, (grad, var) in enumerate(grads_and_vars):
        updated_s_n = s_n[i].assign( (t * grad**2) + (1 - t) * s_n[i] )

        p_n_first = eta * n**(-.5 + delta)
        p_n_second = (1 + tf.sqrt(updated_s_n[i]))**(-1)
        p_n = p_n_first * p_n_second

        updated_var = var.assign_add(-p_n * grad)
        ops.append((updated_s_n[i], p_n_first, p_n_second, p_n, updated_var))
#     increment_n = n.assign_add(1.)
#     ops.append(increment_n)
    return ops

In [34]:
w1 = tf.Variable(tf.constant(1.))
w2 = tf.Variable(tf.constant(2.))
var_list = [w1, w2]

x = tf.constant([3., 4., 5.])
y = tf.constant([.8, .1, .1])

pred = tf.nn.softmax(x * w1 * w2)
loss = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred)))
grads = tf.gradients(loss, var_list)
grads_and_vars = list(zip(grads, var_list))

s_n = tf.Variable(tf.zeros(2))
n = tf.Variable(tf.constant(1.))

train = alp_optimizer_apply_gradients(n, s_n, grads_and_vars)
increment_n = n.assign_add(1.)

In [39]:
sess = tf.InteractiveSession()
init = tf.global_variables_initializer()
sess.run(init)

In [40]:
# starting vals:

# grads_and_vars: [(3.1018744, 1.0), (1.5509372, 2.0)]
# s_n: array([ 0.,  0.], dtype=float32)
# n: 1.0

In [41]:
for _ in range(2):
    the_n, the_s_n, the_grads_and_vars = sess.run([n, s_n, grads_and_vars])
#     print('the_n:', the_n)
#     print('the_s_n:', the_s_n)
    print('the_grads_and_vars:', the_grads_and_vars)
    
#     the_grads_and_vars, results = sess.run([grads_and_vars, train])
    results = sess.run(train)

    print('')
    for result in results:
        s_n_i, p_n_first_i, p_n_second_i, p_n_i, updated_var = result
        print('s_n_i:', s_n_i)
        print('p_n_first_i:', p_n_first_i)
        print('p_n_second_i:', p_n_second_i)
        print('p_n_i:', p_n_i)
        print('updated_var:', updated_var)
        print('')
    sess.run(increment_n)
    

# 
# np.testing.assert_almost_equal(w1_, var_manual[0], 7)
# np.testing.assert_almost_equal(w2_, var_manual[1], 7)

the_grads_and_vars: [(3.1018744, 1.0), (1.5509372, 2.0)]

s_n_i: 0.962162
p_n_first_i: 0.1
p_n_second_i: 0.504821
p_n_i: 0.0504821
updated_var: 0.843411

s_n_i: 0.240541
p_n_first_i: 0.1
p_n_second_i: 0.670939
p_n_i: 0.0670939
updated_var: 1.89594

the_grads_and_vars: [(2.7902498, 0.84341073), (1.241244, 1.8959416)]

s_n_i: 1.6445
p_n_first_i: 0.0712025
p_n_second_i: 0.438139
p_n_i: 0.0311966
updated_var: 0.756364

s_n_i: 0.370555
p_n_first_i: 0.0712025
p_n_second_i: 0.621607
p_n_i: 0.04426
updated_var: 1.841



In [38]:
the_grads_and_vars

[(2.7902498, 0.75636435), (1.241244, 1.8410041)]

In [32]:
# after 1

# s_n[i]: 0.962162479337536
# p_n_first: 0.1
# p_n_second: 0.504821343702
# p_n: 0.0504821343702
# var: 0.84341075974

# s_n[i]: 0.240540619834384
# p_n_first: 0.1
# p_n_second: 0.670938574622
# p_n: 0.0670938574622
# var: 1.89594164057

s_n_i: 0.962162
p_n_first_i: 0.1
p_n_second_i: 0.504821
p_n_i: 0.0504821
updated_var: 0.843411

s_n_i: 0.240541
p_n_first_i: 0.1
p_n_second_i: 0.670939
p_n_i: 0.0670939
updated_var: 1.89594

# after 2

# 2
# s_n[i]: 1.6444956260437862
# p_n_first: 0.07120250977985358
# p_n_second: 0.438139347908
# p_n: 0.0311966212044
# var: 0.756364393664

# s_n[i]: 0.3705552246045456
# p_n_first: 0.07120250977985358
# p_n_second: 0.621607393593
# p_n: 0.0442600065216
# var: 1.84100417304

s_n_i: 1.6445
p_n_first_i: 0.0712025
p_n_second_i: 0.438139
p_n_i: 0.0311966
updated_var: 0.756364

s_n_i: 0.370555
p_n_first_i: 0.0712025
p_n_second_i: 0.621607
p_n_i: 0.04426
updated_var: 1.841
    
# after 3

# s_n[i]: 2.1597428437294326
# p_n_first: 0.05837280797536004
# p_n_second: 0.404922832044
# p_n: 0.0236364827198
# var: 0.694741731383

# s_n[i]: 0.44822725824311604
# p_n_first: 0.05837280797536004
# p_n_second: 0.598982532688
# p_n: 0.0349642923612
# var: 1.80355358733